In [ ]:
%%time

import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-byo"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)


In [ ]:
!conda install -y -c conda-forge xgboost==0.90


Fetch the dataset.

In [ ]:
!aws s3 ls s3://sagemaker-sample-files/datasets/image/MNIST/


In [ ]:
%%time
import pickle
import boto3
import gzip

# Get the data from a public S3
buf=boto3.client('s3').get_object(
    Bucket='sagemaker-sample-files',
    Key='datasets/image/MNIST/mnist.pkl.gz'
)['Body'].read()


# decompress the buffer
decomp_buf = gzip.decompress(buf)
train_set, valid_set, test_set = pickle.loads(decomp_buf, encoding="latin1")


In [ ]:
train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]


Train the XGBClassifier

In [ ]:
import xgboost as xgb
import sklearn as sk

bt = xgb.XGBClassifier(
    max_depth=5, learning_rate=0.2, n_estimators=10, objective="multi:softmax"
)  # Setup xgboost model
bt.fit(train_X, train_y, eval_set=[(valid_X, valid_y)], verbose=False)  # Train it to our data


Save the trained model file

In [ ]:
model_file_name = "DEMO-local-xgboost-model"
bt.save_model(model_file_name)

In [ ]:
!tar czvf model.tar.gz $model_file_name


Upload trained model to S3 bucket.

In [ ]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")


boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)


Import model into Sagemaker

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-2")


In [ ]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])


Create Sagemaker serverless endpoint configuration

In [1]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostServerlessEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 10
            },
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])


DEMO-XGBoostServerlessEndpointConfig-2022-02-01-12-48-58


NameError: name 'sm_client' is not defined

Create Sagemaker endpoint

In [ ]:
%%time
import time

endpoint_name = "DEMO-XGBoostServerlessEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)


Validate the model for use

In [ ]:
runtime_client = boto3.client("runtime.sagemaker")

In [ ]:
import numpy as np

point_X = test_X[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = test_y[0]
np.savetxt("test_point.csv", point_X, delimiter=",")


In [ ]:
%%time
import json


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)
result = response["Body"].read().decode("ascii")
print("Predicted Class Probabilities: {}.".format(result))


In [ ]:
floatArr = np.array(json.loads(result))
predictedLabel = np.argmax(floatArr)
print("Predicted Class Label: {}.".format(predictedLabel))
print("Actual Class Label: {}.".format(point_y))
